<a href="https://colab.research.google.com/github/VeraaaaWei/data_analytics/blob/Python/Fund%20data%20crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import random
import datetime
import re
import numpy as np
import json
from six import StringIO
from six import BytesIO


In [3]:
##获取代理池，详见 https://github.com/1again/SmartProxyPool
def get_proxy():
    data_json = requests.get("http://proxy.1again.cc:35050/api/v1/proxy/?region=中国").text
    data = json.loads(data_json)
    return data['data']['proxy']


def get_url(url, params=None, proxies=None,header=None):
    rsp = requests.get(url, params=params, proxies={"http": proxies},headers=header)    
    rsp.raise_for_status()
    return rsp.text


In [5]:
##获取全基金
url = 'http://fund.eastmoney.com/js/fundcode_search.js'
html = get_url(url,proxies=None)
soup = BeautifulSoup(html, 'html.parser')
exec('securities'+str(soup).strip('var '))
funds = pd.DataFrame(data=securities, index=None, columns=['code','2','name','type','5'])
# fund type类型 股票型-  混合型 债券型 指数型 保本型 理财型 货币型 混合-FOF ...


In [ ]:
def get_header():
 # user_agent列表
    user_agent_list = [
  'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER',
  'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)',
  'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0',
  'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Maxthon/4.4.3.4000 Chrome/30.0.1599.101 Safari/537.36',
  'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.122 UBrowser/4.0.3214.0 Safari/537.36'
    ]
# referer列表
    referer_list = [
  'http://fund.eastmoney.com/110022.html',
  'http://fund.eastmoney.com/110023.html',
  'http://fund.eastmoney.com/110024.html',
  'http://fund.eastmoney.com/110025.html',
  'https://www.baidu.com/s?wd=%E5%A4%A9%E5%A4%A9%E5%9F%BA%E9%87%91%E7%BD%91'
    ]
# 获取一个随机user_agent和Referer
    header = {'User-Agent': random.choice(user_agent_list),'Referer': random.choice(referer_list)}
    return header


def get_fund_data(code, start='', end='',proxy_list=0):
    url = 'http://fund.eastmoney.com/f10/F10DataApi.aspx'
    params = {'type': 'lsjz', 'code': code, 'page': 1, 'per': 40, 'sdate': start, 'edate': end}
    if proxy_list==0:
        html = get_url(url,params,proxies=None,header=get_header())
    else:
        while(1):
            try:
                proxy_random=random.choice(proxy_list)
                html = get_url(url,params,proxies=proxy_random,header=get_header())
                break
            except:
                continue
    soup = BeautifulSoup(html, 'html.parser')
    records = pd.DataFrame(data=None, index=None, columns=['Code','NetAssetValue','AccumulatedNetValue','ChangePercent'])
    tab = soup.findAll('tbody')[0]
    for tr in tab.findAll('tr'):
        if tr.findAll('td') and len((tr.findAll('td'))) == 7:
            date=datetime.datetime.strptime(str(tr.select('td:nth-of-type(1)')[0].getText().strip()),'%Y-%m-%d')
            if tr.select('td:nth-of-type(2)')[0].getText().strip()=='':
                nav=0
            else:
                nav=float(tr.select('td:nth-of-type(2)')[0].getText().strip())

            if tr.select('td:nth-of-type(3)')[0].getText().strip()=='':
                aav=0
            else:
                aav=float(tr.select('td:nth-of-type(3)')[0].getText().strip()) 

            if tr.select('td:nth-of-type(4)')[0].getText().strip('%')=='':
                cpt=0
            else:
                cpt=float(tr.select('td:nth-of-type(4)')[0].getText().strip('%'))
            records.loc[date,:]=[code,nav,aav,cpt]
            
    reg=re.compile(r"(?<=pages:)\d+")
    match=reg.search(str(soup))
    pages=int(match.group(0))
    if pages >1:
        for p in range (2,pages+1):
            params = {'type': 'lsjz', 'code': code, 'page': p, 'per': 40, 'sdate': start, 'edate': end}
            html = get_url(url, params)
            soup = BeautifulSoup(html, 'html.parser')
            tab = soup.findAll('tbody')[0]
            for tr in tab.findAll('tr'):
                if tr.findAll('td') and len((tr.findAll('td'))) == 7:
                    date=datetime.datetime.strptime(str(tr.select('td:nth-of-type(1)')[0].getText().strip()),'%Y-%m-%d')
                    if tr.select('td:nth-of-type(2)')[0].getText().strip()=='':
                        nav=0
                    else:
                        nav=float(tr.select('td:nth-of-type(2)')[0].getText().strip())
                    
                    if tr.select('td:nth-of-type(3)')[0].getText().strip()=='':
                        aav=0
                    else:
                        aav=float(tr.select('td:nth-of-type(3)')[0].getText().strip()) 
                    
                    if tr.select('td:nth-of-type(4)')[0].getText().strip('%')=='':
                        cpt=0
                    else:
                        cpt=float(tr.select('td:nth-of-type(4)')[0].getText().strip('%'))
                        
                    records.loc[date,:]=[code,nav,aav,cpt]    
    return records




## 获取数据示例：get_fund_data('163402','2018-09-18','2019-09-18')


In [9]:

get_fund_data('163402','2018-09-18','2019-09-18')


,Code,NetAssetValue,AccumulatedNetValue,ChangePercent
2019-09-18,163402,0.7928,9.659,0.69
2019-09-17,163402,0.7874,9.6374,-1.19
2019-09-16,163402,0.7969,9.6754,-0.15
2019-09-12,163402,0.7981,9.6802,0.88
2019-09-11,163402,0.7911,9.6522,-0.49
...,...,...,...,...
2018-09-25,163402,0.6421,9.0571,-0.5
2018-09-21,163402,0.6453,9.0699,2.02
2018-09-20,163402,0.6325,9.0188,-0.28
2018-09-19,163402,0.6343,9.026,1.18
